# Support Vectore Machine


In [1]:
import math
import pickle
import gzip
import numpy as np
import pandas
import matplotlib.pylab as plt
%matplotlib inline

[50 Points] Problem 1 - Basic concepts of SVM
---

**Part A**: 
* What are the main differences between the primal and the dual representations?

* For $\xi$, $C$, $\alpha$, and $\beta$, what is their role and if there is a special value what is the value and what does it mean?

**In the primal problem, we are minimizing the objective function with regard to $w, b, \xi$. In this representaion, we solve directly for $w$ and $b$.**

**The dual problem is a different perspective on the primal in which take the langrangian representation of the primal problem and turn in from a min-max problem to a max-min problem, which is allowed under strong duality. In the dual problem, we are maximixing a different objective function with regard to $\alpha$. Notice, this dual objective function does not have $w$ or $b$ at all. Instead, we find the maximizing $\alpha$ value and use the KKT conditions to determine $w$ and $b$.**

**$\xi$: slack variable, how "wrong" a classifed point is. Special values:**

- $\xi_i$ = 0 when i is at least 1 margin distance from the decision boundary on the correct side. 
- $\xi_i$ = 2 when i is 1 margin distance from the decision boundary on the incorrect side.

**$C$: the tradeoff between the margin and slack variable terms. It essentially means "how much should misclassified points affect the model". Special values:**

- $C$ = $\infty$ means there is an infinite penalty for missclassified points, which essentially turns soft-margin SVM into hard-margin
- $C$ = 0 is not exactly a "special value", but it would mean that you don't care at all if points are misclassified, which defeats the point of a classifier

**$\alpha$: a KKT multiplier that, in the primal lagrangian, enforces the $y_i(w^Tx_i + b) - 1 + \xi_i \geq 0$ primal condition and therefore, in the converse dual problem, the value when want to maximize with. Special values:**

It helped me to think of $\alpha$ in the lagrangian formulation as an adversarial term. If our condition above was not met (i.e $y_i(w^Tx_i + b) - 1 + \xi_i < 0$), $\alpha$ could equal infinity, making this term huge when we should be minimizing. Conversely, if the condition above is met, $\alpha$ equal to 0 would be the best adversarial option.

**$\beta$: a KKT multiplier that, in the primal lagrangian, enforces the $\xi_i \geq 0$ primal condition, similar to the $\alpha$ above**



**PART B**: 

 * Given a weight vector, implement the *find_support* function that returns the indices of the support vectors.
 * Given a weight vector, implement the *find_slack* function that returns the indices of the vectors with nonzero slack.
 * Given the alpha dual vector, implement the *weight_vector* function that returns the corresponding weight vector.

In [2]:
import numpy as np

kINSP = np.array([(1, 8, +1),
               (7, 2, -1),
               (6, -1, -1),
               (-5, 0, +1),
               (-5, 1, -1),
               (-5, 2, +1),
               (6, 3, +1),
               (6, 1, -1),
               (5, 2, -1)])

kSEP = np.array([(-2, 2, +1),    # 0 - A
              (0, 4, +1),     # 1 - B
              (2, 1, +1),     # 2 - C
              (-2, -3, -1),   # 3 - D
              (0, -1, -1),    # 4 - E
              (2, -3, -1),    # 5 - F
              ])


def weight_vector(x, y, alpha):
    """
    Given a vector of alphas, compute the primal weight vector w.
    The vector w should be returned as an Numpy array.
    """

    w = np.zeros(len(x[0]))
    for i in range(len(x)):
        w += x[i] * y[i] * alpha[i]
    return w



def find_support(x, y, w, b, tolerance=0.001):
    """
    Given a set of training examples and primal weights, return the indices
    of all of the support vectors as a set.
    """
    support = set([i for i in range(len(x)) if abs(y[i]*(np.dot(w,x[i]) + b) - 1) <= tolerance])
    return support



def find_slack(x, y, w, b):
    """
    Given a set of training examples and primal weights, return the indices
    of all examples with nonzero slack as a set.
    """

    slack = set([i for i in range(len(x)) if (y[i] * (np.dot(w, x[i]) + b)) < 0])
    return slack

In [3]:
%run -i tests/tests.py

....
----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


**PART C**

The goal of this problem is to correctly classify test data points, given a training data set.
For this problem, assume that we are training an SVM with a quadratic kernel– that is, our kernel function is a polynomial kernel of degree 2. You are given the data set presented in Figure 1. The slack penalty C will determine the location of the decision boundary.

Justify the following questions in a sentence or via drawing decision boundary.
![training_data](./data/data.png)

* Where would the decision boundary be for very large values of C ?
* Where you would expect the decision boundary to be if  C = 0 ?
* Which of the two cases above would you expect to generalize better on test data? Why?

**1. I would expect the boundary for very large values of C to be a parabola (due to degree 2) that wraps around the green points. This is because the high slack penalty means the model will shift the decision boundary to avoid a high penalty for misclassifiying the two outlying red points, even at the cost of reducing the margin (high variance, low bias)**

**2. I woulc expect the boundary for C = 0 (or in reality it would likely be very close to 0) to pass through the large gap between the majority red and green clusters, and thus misclassifying the two red outlying red points. This is because a low slack penalty means the model will care more about maximizing the margin than missclassifying some points (high bias, low variance)**

**3. In this example, I would expect the low C to generalize better on test data. A very low C will roughly classify data in the bottom left corner as red and the top right corner  as green, even though a few outliers could be misclassified. A high C will overfit to the training data, and as a result would assume only points in the very upper right corner of this plot are green, likely misclassifying more points in the test data.**

[50 points] Problem 2 -- SVM with Sklearn
---

In this problem, you are going to get familiar with important practical functions in scikit-learn such as pipeline, grid search, and cross validation. You will experiment with these using support vector machines.

Note that grid search can take some time on your laptop, so make sure that your code is correct with a small subset of the training data and search a reasonable number of options.

* Use the Sklearn implementation of support vector machines to train a classifier to distinguish Positive and negative sentiments
* Experiment with linear, polynomial, and RBF kernels. In each case, perform a GridSearch to help determine optimal hyperparameters for the given model (e.g. C for linear kernel, C and p for polynomial kernel, and C and  for RBF). Comment on the experiments you ran and optimal hyperparameters you found.
Hint: http://scikit-learn.org/stable/modules/grid_search.html
* Comment on classification performance for each model for optimal parameters by testing on a hold-out set.

Following is a dataset containing reviews and sentiments associated with it.

Create a SVM Classifier to predict positive or negative sentiments

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
reviews  = pd.read_csv('./data/reviews.csv')
train, test = train_test_split(reviews, test_size=0.2, random_state=4622)
X_train = train['reviews'].values
X_test = test['reviews'].values
y_train = train['sentiment']
y_test = test['sentiment']

In [5]:
len(X_train),sum(y_train),len(X_test),sum(y_test)

(4000, 2004, 1000, 496)

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score

**PART A**

Use CountVectorizer to vectorize reviews as dictionary of term frequencies.
Define the crossvalidation split using StratifiedKFold.

In [12]:
def tokenize(text): 
    tknzr = TweetTokenizer()
    return tknzr.tokenize(text)

en_stopwords = set(stopwords.words("english")) 

# CREATE CountVectorizer using sklearn.feature_extraction.text.CountVectorizer
# Hint: use the above tokenize function
# Hint: play with different parameters, in particular, min_df can help with generalizability
# YOUR CODE HERE
count_vect = CountVectorizer(tokenizer=tokenize, stop_words=en_stopwords, min_df=5)
# split dataset using StratifiedKFold into 5 splits using sklearn.model_selection.StratifiedKFold.
# YOUR CODE HERE
skf = StratifiedKFold(n_splits=5)
cv = skf.get_n_splits(X_train, y_train)

**PART B**
* Create pipeline with Count Vectorizer and SVM Classifier
* Define grid search parameters
* Create GridSearchCV object with pipeline created and fit the data.
* Compute accuracy on best estimator from GridSearchCV

In [29]:
# DEFINE GRID SEARCH PARAMETERS kernel, C, degree, gamma respectively
# YOUR CODE HERE
svc=SVC()
c_range = np.logspace(-8, 3, 5, base=2)
gamma_range = np.logspace(-5, 3, 3, base=2)
degree_range=np.linspace(2.0,5.0, num=3)
param_grid = dict(svc__gamma=gamma_range, svc__C=c_range, svc__kernel=('poly', 'rbf'), svc__degree=degree_range)

In [30]:
print(c_range, gamma_range)

[3.90625000e-03 2.62780130e-02 1.76776695e-01 1.18920712e+00
 8.00000000e+00] [0.03125 0.5     8.     ]


In [31]:
np.random.seed(1234)
# Define pipeline using make_pipeline with vectorizer and SVM Classifier
# YOUR CODE HERE
pipeline = make_pipeline(count_vect, svc)

# Create GridSearchCV with pipeline and grid search parameters, scoring as accuracy.
# for example grid_svm = GridSearchCV(pipeline, param_grid, cv, scoring="accuracy")

# YOUR CODE HERE
grid_svm = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=cv, scoring="accuracy", verbose=1)

# For debugging purposes, it makes sense to use a smaller set of training set to speed up the grid search progress
grid_svm.fit(X_train[0:50], y_train[0:50])

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 450 out of 450 | elapsed:   27.4s finished
/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=...f', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'svc__gamma': array([0.03125, 0.5    , 8.     ]), 'svc__C': array([3.90625e-03, 2.62780e-02, 1.76777e-01, 1.18921e+00, 8.00000e+00]), 'svc__kernel': ('poly', 'rbf'), 'svc__degree': array([2. , 3.5, 5. ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [32]:
print("best params:")
print(grid_svm.best_params_)

print("best cv score:")
print(grid_svm.best_score_)

best params:
{'svc__C': 0.026278012976678578, 'svc__degree': 3.5, 'svc__gamma': 0.03125, 'svc__kernel': 'poly'}
best cv score:
0.68


best params:
{'svc__C': 0.03125, 'svc__degree': 1.0, 'svc__gamma': 0.03125, 'svc__kernel': 'linear'}
best cv score:
0.87025

In [52]:
def report_results(model, X, y):
    pred = model.predict(X)        
    acc = accuracy_score(y, pred)
    f1 = f1_score(y, pred)
    prec = precision_score(y, pred)
    rec = recall_score(y, pred)
    result = {'f1': f1, 'acc': acc, 'precision': prec, 'recall': rec}
    return result

In [53]:
report_results(grid_svm.best_estimator_, X_test, y_test)

{'f1': 0.4967320261437908,
 'acc': 0.615,
 'precision': 0.7063197026022305,
 'recall': 0.38306451612903225}

**PART C**

Explain the overall procedure and report the final result that you obtain including which kernel and hyperparameter was chosen.

The overall procedure is to vectorize the text in the dataset, excluding stop words and words that occur in less than 5% of the overall dataset (thank to min_df). Then we fit an SVM model using 5 fold cross-validation on the vectorized data with every combination of paramters in our parameter grid (C, gamma, degree, kernel) to determine which were the best to use.

In my case the best were:

* **C**: 0.021262343752724643

* **Gamma**: 0.03125

* **Degree**: 2

* **Kernel**: polynomial